# Descrizione notebook
In questo notebook si fa utilizzo dello User-Based Collaborative Filtering per raccomandare una barzelletta ad un utente.

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
import math

In [2]:
# lettura dei rating normalizzati dal relativo CSV
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')

# lettura dei rating non normalizzati dal relativo CSV (serve solo per calcolare la predizione, nella
# quale serve calcolare il rating medio dell'utente)
# non c'è bisogno di levare i 99, tanto questo df serve solo per calcolare il rating medio dell'utente target
# e si droppano le barzellette che non ha votato
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')

# Similarità tra utenti: Coefficiente di Pearson

In [3]:
# funzione per trovare la similarità tra due utenti: Coefficiente di Pearson
def weight_factor(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        if not(math.isnan(i)) and not(math.isnan(j)):
            t1+=i*j # numeratore
            t2+=i*i # denominatore_1
            t3+=j*j # denominatore_2
    return t1 / (np.sqrt(t2) * np.sqrt(t3))

# Predizione
Assegnazione degli score per le barzellette non ancora valutate dall'utente target.

In [4]:
def score_user_item(item_id, neighbours_df, neighbour_user_similarity, active_user_mean_rating):
    item_rating = neighbours_df[item_id]
    t1, t2 = 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity # numeratore
        t2+= similarity # denominatore
    score = active_user_mean_rating + t1 / t2
    return score

# Raccomandazione

In [5]:
def recommendation_1(active_user_id, normalized_ratings_df, ratings_df):
    # si prende la riga relativa all'id dell'utente
    active_user = normalized_ratings_df[normalized_ratings_df['user_id'] == active_user_id]
    # della riga si considerano solo i rating espressi dall'utente
    active_user_rating = active_user.iloc[:, 2:]
    # si droppa altrimenti con lui uscirebbe similarità massima (= 1)
    normalized_ratings_df = normalized_ratings_df.drop(normalized_ratings_df[normalized_ratings_df['user_id'] == active_user_id].index)
    # vengono messi i rating dell'utente target in una lista,
    # ravel() esempio: [[1,2],[3,4]] = [1,2,3,4]
    active_user_rating_list = active_user_rating.values.ravel()
    # si calcola la similarità tra l'utente target e tutti gli altri utenti
    similarity = []
    for i in range(normalized_ratings_df.shape[0]):
          similarity.append([normalized_ratings_df.iloc[i, 0], weight_factor(active_user_rating_list, normalized_ratings_df.iloc[i, 2:])])
    # si crea un dataframe
    similarity = pd.DataFrame(similarity, columns = ['user_id', 'similarity'])
    # si buttano i vicini che sono troppo poco simili
    # similarity = similarity[similarity['similarity'] > 0.1]
    # si prendono le barzellette raccomandabili all'utente
    recommendation_columns = []
    for column in active_user_rating.columns:
        if math.isnan(active_user_rating[column].values[0]):
            recommendation_columns.append(column)
    # si calcola la media delle valutazioni dello user target in modo 
    # da avere una predizione più precisa (formula vista nel corso)
    active_user_raw_ratings = ratings_df[ratings_df['user_id'] == active_user_id].iloc[:, 2:]
    active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
    # si prendono i rating dei neighbor dell'utente target
    neighbours_df = normalized_ratings_df[normalized_ratings_df['user_id'].isin(similarity["user_id"])]
    # si selezionano solo le barzellette che si volgiono raccomandare all'utente (quelle che non ha valutato)
    neighbours_df = neighbours_df[['user_id'] + recommendation_columns]
    # raccomandazione della barzelletta con la predizione più alta
    top_score = -np.inf
    joke_to_suggest = ''
    final = []
    # per ogni barzellette che si potrebbe raccomandare si calcola lo score
    for column in recommendation_columns:
        # si devono prendere i 30 vicini più simili che hanno valutato la barzelletta che si vuole raccomandare
        app = neighbours_df[neighbours_df[column].notna()][["user_id", column]]
        # si fa un join con la matrice di similarità sulla base dello user_id
        app = pd.merge(app, similarity, on = "user_id")
        # si ordina in ordine decrescente per la similarità e si prendono i primi 30 elementi
        app = app.sort_values(by=['similarity'], ascending=False).head(30)
        # si predice la valutazione dell'utente in base ai suoi vicini
        score = score_user_item(column, app, app["similarity"], active_user_mean_rating)
        print("per la barzelletta " + str(column) + " si è predetto un punteggio di: " + str(score))
        if score > top_score:
            top_score = score
            joke_to_suggest = column
        app_2 = {}
        app_2["user_id"] = active_user_id
        app_2[column] = score
        final.append(app_2)
    print('\nil valore più alto predetto è ' + str(top_score) + ' su 10')
    print('quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è: ', joke_to_suggest) 
    return final

In [6]:
recommendation_1(1001, normalized_ratings_df, ratings_df)

per la barzelletta joke_1 si è predetto un punteggio di: 4.946422247845393
per la barzelletta joke_2 si è predetto un punteggio di: 4.231802699247759
per la barzelletta joke_3 si è predetto un punteggio di: 4.6167440703613325
per la barzelletta joke_4 si è predetto un punteggio di: 4.711268629409112
per la barzelletta joke_9 si è predetto un punteggio di: 4.057639500844513
per la barzelletta joke_23 si è predetto un punteggio di: 5.0954744109093095
per la barzelletta joke_24 si è predetto un punteggio di: 2.858749823294116
per la barzelletta joke_33 si è predetto un punteggio di: 4.766254653547798
per la barzelletta joke_37 si è predetto un punteggio di: 4.618399335229611
per la barzelletta joke_43 si è predetto un punteggio di: 3.9585039203072085
per la barzelletta joke_47 si è predetto un punteggio di: 6.6123087727683805
per la barzelletta joke_55 si è predetto un punteggio di: 5.25092530575441
per la barzelletta joke_57 si è predetto un punteggio di: 1.564840717900211
per la barzell

[{'user_id': 1001, 'joke_1': 4.946422247845393},
 {'user_id': 1001, 'joke_2': 4.231802699247759},
 {'user_id': 1001, 'joke_3': 4.6167440703613325},
 {'user_id': 1001, 'joke_4': 4.711268629409112},
 {'user_id': 1001, 'joke_9': 4.057639500844513},
 {'user_id': 1001, 'joke_23': 5.0954744109093095},
 {'user_id': 1001, 'joke_24': 2.858749823294116},
 {'user_id': 1001, 'joke_33': 4.766254653547798},
 {'user_id': 1001, 'joke_37': 4.618399335229611},
 {'user_id': 1001, 'joke_43': 3.9585039203072085},
 {'user_id': 1001, 'joke_47': 6.6123087727683805},
 {'user_id': 1001, 'joke_55': 5.25092530575441},
 {'user_id': 1001, 'joke_57': 1.564840717900211},
 {'user_id': 1001, 'joke_58': 0.08426996774378281},
 {'user_id': 1001, 'joke_60': 2.3602276388425345},
 {'user_id': 1001, 'joke_64': 4.170134824607456},
 {'user_id': 1001, 'joke_67': 4.75233318890423},
 {'user_id': 1001, 'joke_71': 1.3164611344901287},
 {'user_id': 1001, 'joke_72': 5.401471921650529},
 {'user_id': 1001, 'joke_73': 5.586445365352631},